In [1]:
from osgeo import gdal
import numpy
from numpy.core.umath import add, subtract


class MDimage(object):
    def __init__(self, filepath):
        self.filepath = filepath
        self.ds = gdal.Open(filepath, gdal.GA_ReadOnly)
        self.iminfo = dict()
        self.iminfo['bandnum'] = self.ds.RasterCount
        self.iminfo['cols'] = self.ds.RasterXSize
        self.iminfo['rows'] = self.ds.RasterYSize
        self.iminfo['originX'] = self.ds.GetGeoTransform()[0]
        self.iminfo['originY'] = self.ds.GetGeoTransform()[3]
        self.iminfo['pixelWidth'] = self.ds.GetGeoTransform()[1]
        self.iminfo['pixelHeight'] = self.ds.GetGeoTransform()[5]

class Index(MDimage):
    def __init__(self, filepath):
        self.filepath = filepath
        super(Index, self).__init__(filepath)

    def Band2Array(self):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        array = band.ReadAsArray(0, 0, cols, rows)
        #array= array.reshape(cols*rows)
        return  array
 
    def WriteArrayAsImage(self, out_fname, outArray):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        driver = self.ds.GetDriver()
        self.outArray = outArray
        outArray= outArray.reshape([rows,cols])
        outDS = driver.Create(out_fname, cols, rows, 1, gdal.GDT_Float32)
        outDS.SetGeoTransform(self.ds.GetGeoTransform())
        outDS.SetProjection(self.ds.GetProjection())
        outBand = outDS.GetRasterBand(1)
        outBand.WriteArray(outArray)
        outDS = None
        del outDS, outBand



In [2]:
path="/home/faizan/Desktop/full/"
limage = Index(path+"A2008153.US.machineflood.tif")
# Get a pixel column number
cols = limage.iminfo['cols']
# Get pixel row number
rows = limage.iminfo['rows']
# Get pixel data as array
BS = limage.Band2Array()



In [3]:
BS

array([[   0.,    0.,    0., ..., -999., -999., -999.],
       [-999.,    0.,    0., ..., -999., -999., -999.],
       [-999., -999.,    0., ..., -999., -999., -999.],
       ..., 
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.],
       [-999., -999., -999., ..., -999., -999., -999.]], dtype=float32)

In [27]:
import scipy
from scipy import misc
#scipy.misc.imsave((path+"PNG.png"), BS)
scipy.misc.toimage(BS,  cmax= 1.0, cmin=0.0,).save(path+ "PNG1.png")
#convert -background black -opaque '#939393' (path+ "PNG1.png") (path+ "PNG2.png")

In [11]:
PN= scipy.misc.imread(path+ "PNG1.png")

In [20]:
PN

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [19]:
P

(array([   0,    0,    0, ..., 1999, 1999, 1999]),
 array([ 175,  176,  177, ..., 4416, 4417, 4418]))

In [6]:
import matplotlib.pyplot as plt
#%matplotlib inline
plt.imshow(BS)
plt.savefig(path+"PNG1.png")

In [7]:
path="/home/faizan/Desktop/full/"
with open('/home/faizan/USA_data/PALSAR/corr/780/list.txt',"r") as f:
    #g=len(f.readlines())     #print sum(1 for _ in f)
    for line in f:
        l= line.split(".")
        #assigning names
        bs=path+str(l[0])+"."+str(l[1])+".backscatter_try.tif"
        flood=path+str(l[0])+"."+str(l[1])+".flood.tif"

        # loading images
        #red band
        limage = Index(bs)
        # Get a pixel column number
        cols = limage.iminfo['cols']
        # Get pixel row number
        rows = limage.iminfo['rows']
        # Get pixel data as array
        BS = limage.Band2Array()

        if str(l[0]) == "HH":
            #calculate Flood
            Flood= (limage.CalcFlood_HH(BS))
        else:
            Flood= (limage.CalcFlood_HV(BS))
        #Get Flood as tiff
        limage.WriteArrayAsImage(flood, Flood)
        del Flood, BS
f.close()


